In [2]:
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
os.listdir('data')

['bmr15315_1.out', 'bmr17160_1.out', 'bmr7339_1.out']

In [4]:
in_df = pd.read_csv('data/bmr15315_1.out', sep = ' ', names=['Residue', 'ShiftCript index'])

In [5]:
index_tensor = torch.tensor(in_df['ShiftCript index'])

In [6]:
windows = index_tensor.unfold(0, 6, 1)
# print(windows)

In [130]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
#         self.conv1 = nn.Conv1d(6, 36, 2)
        self.linear1 = nn.Linear(6, 10)
        self.linear2 = nn.Linear(10, 1)
        
    def forward(self, x):
#         x = F.max_pool1d(F.relu(self.conv1(x)),2)
        x = F.relu(self.linear1(x))
        print(x)
        x = self.linear2(x)
        return x
    
net = Net()
print(net)

Net(
  (linear1): Linear(in_features=6, out_features=10, bias=True)
  (linear2): Linear(in_features=10, out_features=1, bias=True)
)


In [32]:
# For some reason 1D convolution and I don't like each other

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(1, 12, 3)  # (in_channels, out_channels, kernel_size)  # DON'T TOUCH FOR NOW
        self.linear1 = nn.Linear(12, 6)
        self.linear2 = nn.Linear(6, 1)
        
    def forward(self, x):
        x = F.max_pool1d(F.relu(self.conv1(x)),4)
        print(x)
#         print(x.size())
        x = x.view(-1, 12)  # Flatten!!! Eureka!!!
        print(x)
        x = F.relu(self.linear1(x))
        x = self.linear2(x)
        return x
    
net = Net()
print(net)

Net(
  (conv1): Conv1d(1, 12, kernel_size=(3,), stride=(1,))
  (linear1): Linear(in_features=12, out_features=6, bias=True)
  (linear2): Linear(in_features=6, out_features=1, bias=True)
)


In [89]:
# Now with 2 targets, and tuneable hyperparameters
number_of_kernels_conv = 42
# These 2 not fully understood. These values work. 
kernel_size = 3
pooling_window_size = 3

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(1, int(number_of_kernels_conv), int(kernel_size))  # (in_channels, out_channels, kernel_size)  # DON'T TOUCH FOR NOW
        self.linear1 = nn.Linear(int(number_of_kernels_conv * kernel_size / pooling_window_size) , 6)
        self.linear2 = nn.Linear(6, 1)
        self.linear_alpha = nn.Linear(int(number_of_kernels_conv * kernel_size / pooling_window_size),8)
        self.linear_beta = nn.Linear(8,4)
        self.linear_gamma = nn.Linear(4,1)
        
    def forward(self, x):
        x = F.max_pool1d(F.relu(self.conv1(x)), int(pooling_window_size))
#         print(x)
        print(x.size())
        x = x.view(-1, int(number_of_kernels_conv * kernel_size / pooling_window_size))  # Flatten!!! Eureka!!!
        print(x)
        y = F.relu(self.linear_alpha(x))
        y = F.relu(self.linear_beta(y))
        y = self.linear_gamma(y)
        x = F.relu(self.linear1(x))
        x = self.linear2(x)
        return x, y
    
net = Net()
print(net)

Net(
  (conv1): Conv1d(1, 42, kernel_size=(3,), stride=(1,))
  (linear1): Linear(in_features=42, out_features=6, bias=True)
  (linear2): Linear(in_features=6, out_features=1, bias=True)
  (linear_alpha): Linear(in_features=42, out_features=8, bias=True)
  (linear_beta): Linear(in_features=8, out_features=4, bias=True)
  (linear_gamma): Linear(in_features=4, out_features=1, bias=True)
)


In [109]:
# print(list(net.named_parameters()))

In [18]:
# a = windows[0].reshape(-1,1).unsqueeze(0)
a = windows[0]
print(a)

tensor([-10.0000,   0.6654,   0.5337,   0.6093,   0.9774,   0.6371])


In [44]:
a = torch.randn(1,1,6)  # (Batch size, channels, size of input)
print(a)

tensor([[[ 1.2301, -0.0707,  0.6961,  0.0746,  1.4741, -1.2506]]])


In [9]:
# windows

In [56]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight
# print(params)

12
torch.Size([10, 1, 3])


In [90]:
net(a)

torch.Size([1, 42, 1])
tensor([[0.2259, 0.0970, 0.5192, 0.3621, 0.0000, 0.5367, 0.9599, 0.4111, 1.1249,
         0.7193, 0.6375, 0.7460, 0.0000, 0.0000, 0.1372, 0.0000, 1.0639, 0.3342,
         0.1586, 0.2716, 0.0000, 0.0000, 0.2195, 0.7708, 0.3733, 0.8270, 0.5340,
         0.8092, 0.3613, 0.4693, 0.3160, 0.5514, 0.0000, 0.0052, 0.9102, 1.1425,
         0.9064, 0.3092, 0.0000, 0.1073, 0.0000, 0.5520]],
       grad_fn=<ViewBackward>)


(tensor([[0.4058]], grad_fn=<AddmmBackward>),
 tensor([[0.5610]], grad_fn=<AddmmBackward>))